In [1]:
import pandas as pd
import pandas.io.data as web
import datetime
import statsmodels.formula.api as sm
import numpy as np
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
from scipy.optimize import curve_fit
plotly.tools.set_credentials_file(username='dev.alex.ckl.1834', api_key='emoFXfJedARSI55osKBo')

D:\Files\Programing and Data\Anaconda\lib\site-packages\pandas\io\data.py:35: FutureWarning: 
The pandas.io.data module is moved to a separate package (pandas-datareader) and will be removed from pandas in a future version.
After installing the pandas-datareader package (https://github.com/pydata/pandas-datareader), you can change the import ``from pandas.io import data, wb`` to ``from pandas_datareader import data, wb``.
  FutureWarning)


In [2]:
start = datetime.datetime(1970,1,1)

In [3]:
end = datetime.date.today()

In [4]:
cpi = web.DataReader("CPIAUCSL", "fred", start, end)

In [5]:
unr = web.DataReader("UNRATE", "fred", start, end)

In [6]:
inflation_t = cpi.apply(lambda x: x / x[0])

In [7]:
trace_inf = go.Histogram(x=inflation_t["CPIAUCSL"])
data_inf = [trace_inf]
py.iplot(data_inf)

In [8]:
pc = pd.DataFrame({"UnRate": unr["UNRATE"],
                    "Inf": inflation_t["CPIAUCSL"]})

In [9]:
pc.tail()

,Inf,UnRate
DATE,,
2016-09-01,6.358997,4.9
2016-10-01,6.377150,4.8
2016-11-01,6.390475,4.6
2016-12-01,6.406887,4.7
2017-01-01,6.442164,4.8


In [10]:
pc_model = sm.ols("inflation_t ~ UnRate", data=pc).fit()

In [11]:
print(pc_model.summary())

                            OLS Regression Results                            
Dep. Variable:            inflation_t   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.714
Date:                Wed, 22 Feb 2017   Prob (F-statistic):              0.100
Time:                        23:17:59   Log-Likelihood:                -1084.2
No. Observations:                 565   AIC:                             2172.
Df Residuals:                     563   BIC:                             2181.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept      4.1970      0.295     14.209      0.0

In [25]:
tracepc = go.Scatter(
                    x=unr["UNRATE"], 
                    y=inflation_t["CPIAUCSL"],
                    mode = "markers", 
                     marker = dict(size= 12,
                                   color = np.random.randn(1000),
                                   opacity= 0.9)
                            )
datapc = [tracepc]
layoutpc = go.Layout(title = 'Phillips Curve', titlefont=dict(size='24'),
                  xaxis=dict(
                            title='Unemployment %',
                            titlefont=dict(family='Times New Roman', size='18'),
                            ticklen=5),
                  yaxis=dict(
                            title='Inflation %',
                            titlefont=dict(family='Arial', size='18'),
                            ticklen=5),
                    height=900)
figure = go.Figure(data = datapc, layout=layoutpc)
py.iplot(figure, filename='phillips')

#This is the best Philips Curve I was able to recreate using Data
#You can see the stratification of the different Curves!

In [ ]:
# Testing a polynomial fit
nppc = np.array(pc)

In [ ]:
x=nppc[:,0]
y=nppc[:,1]

In [ ]:
z = np.polyfit(x,y,3)
f = np.poly1d(z)

In [ ]:
print(f)

In [ ]:
x_new = np.linspace(x[0], x[-1], 50)
y_new = f(x_new)

In [ ]:
trace1 = go.Scatter(
                  x=y,
                  y=x,
                  mode='markers',
                  marker=go.Marker(color='rgb(255, 127, 14)'),
                  name='Data'
                  )

trace2 = go.Scatter(
                  x=y_new,
                  y=x_new,
                  mode='lines',
                  marker=go.Marker(color='rgb(31, 119, 180)'),
                  name='Fit'
                  )

annotation = go.Annotation(
                  x=6,
                  y=-4.5
                  showarrow=False
                  )
layout = go.Layout(
                title='Phillips test fit',
                plot_bgcolor='rgb(229, 229, 229)',
                  xaxis=go.XAxis(zerolinecolor='rgb(255,255,255)', gridcolor='rgb(255,255,255)'),
                  yaxis=go.YAxis(zerolinecolor='rgb(255,255,255)', gridcolor='rgb(255,255,255)'),
                  annotations=[annotation]
                )

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)

py.plot(fig, filename='Phillpis test fit')
# It doesnt really work

In [ ]:
# testing an exponential fit
i = np.array(inflation_t[-50:])
u = np.array(unr[-50:])

In [ ]:
def exponential_func(x, a, b, c):
    return np.array(a*np.exp(-b*x)+c)

In [ ]:
popt, pcov = curve_fit(exponential_func, u, i, p0=None)

In [ ]:
help(curve_fit)